In [1]:
import random

import tensorflow as tf
from tensorflow.keras import layers
import keras
import numpy as np
import math

In [2]:
def gen_g_data(n_dim, n_sample=100000):
    c = np.ones(n_dim)
    var_0 = np.identity(n_dim)

    ori = np.random.multivariate_normal(c, var_0, n_sample)
    return ori

In [3]:
def gen_n_dim_gaussian(n_dim, lam = 1, n_sample = 100000):
    mean = np.zeros(n_dim)
    var = np.identity(n_dim)

    return np.random.multivariate_normal(mean, lam * var, n_sample)

In [4]:
def d_real_loss(real, fake):
    """
    calculate the expectation of
        real + 1 - fake

    :param real:
    :param fake:
    :return:
    """
    return tf.reduce_mean(real) + tf.reduce_mean(1. - fake)

In [5]:
def d_metric(ori, contrast):
    # get the probabilities
    ori = Discriminator(ori, training=False)
    contrast = Discriminator(contrast, training=False)

    # get D^
    ori = tf.where(ori < 0.5, 0., 1.)
    contrast = tf.where(contrast < 0.5, 0., 1.)

    # tf.print("true-positive", tf.reduce_sum(ori))
    # tf.print("false-positive", tf.reduce_sum(contrast))

    # print calculated TVD
    a = d_real_loss(ori, contrast) - 1
    # tf.print("TVD-ori", metric, "TVD-cal", a, "delta", tf.abs(metric - a))
    return a

In [6]:
def inv_sphere_proj(raw_point, n_dim, n_train, radius):
    """inverse stereographic projection

    n_dim:
    the dimension of the original data

    n_train:
    the number of points to be projected

    radius:
    the radius of the sphere
    """

    res = []
    for i in range(n_train):
        tmp = []
        normal = tf.sqrt(tf.reduce_sum(tf.square(raw_point[i])))
        for j in range(n_dim):
            tmp.append(tf.reduce_sum((2 * raw_point[i][j] / (normal + 1) * radius)))
        tmp.append(tf.reduce_sum(((normal - 1) / (normal + 1) * radius)))
        res.append(tf.stack(tmp))
    res = tf.stack(res)
    return res

In [7]:
n_sample = 1000

In [8]:
Generator = keras.models.load_model("model/c=0.5epoch=15/generator.h5")
Discriminator = keras.models.load_model("model/c=0.5epoch=15/discriminator.h5")
for i in range(50):
    test_data = gen_g_data(10, n_sample)
    gen_data = gen_n_dim_gaussian(10, 1, n_sample)
    input_data = Generator(gen_data)
    sphPoints = inv_sphere_proj(input_data, 9, n_sample, math.sqrt(10))
    fake_test_data = sphPoints + gen_n_dim_gaussian(10, 1.5, n_sample)
    tf.print((d_metric(test_data, fake_test_data) - 0.343927681) / 0.343927681)

Metal device set to: Apple M1 Max


2022-09-10 18:35:19.053271: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-10 18:35:19.053387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


0.0525471717
0.0263786949
0.0903455
0.0990683287
0.0787153
0.125236794
0.0205635931
0.305507362
-0.0172350835
0.0816230252
0.0787153
0.104883775
0.0438243449
0.0729002
0.174665675
0.0903458446
0.122329079
-0.0666639656
-0.0492186584
0.0903455
0.0554545484
-0.00560488133
0.0612696484
0.128144175
0.0699924752
-0.0230505317
-0.00269715744
-0.0521260388
0.0554545484
0.0845307484
0.0525468215
0.101976052
0.0670850947
0.0932532251
0.165942863
0.0351015218
0.00893339142
0.0961609483
0.00021056649
0.0729002
0.0118407691
0.0932535678
0.0321937948
0.0438243449
0.128144175
0.0176558699
0.0641777217
0.0641773716
0.0176562164
0.00893339142


In [9]:
Generator = keras.models.load_model("model/c=1epoch=15/generator.h5")
Discriminator = keras.models.load_model("model/c=1epoch=15/discriminator.h5")
for i in range(50):
    test_data = gen_g_data(10, n_sample)
    gen_data = gen_n_dim_gaussian(10, 1, n_sample)
    input_data = Generator(gen_data)
    sphPoints = inv_sphere_proj(input_data, 9, n_sample, math.sqrt(10))
    fake_test_data = sphPoints + gen_n_dim_gaussian(10, 2, n_sample)
    tf.print((d_metric(test_data, fake_test_data) - 0.552562714) / 0.552562714)

-0.0100670261
-0.0516910478
-0.0480718128
-0.0571203344
-0.0173061453
0.0152695701
-0.0462619774
-0.0426425263
-0.0444521457
-0.0498814322
-0.00101828773
-0.0498814322
0.00441099843
-0.00282790605
-0.0535008833
-0.0173059311
0.0387963317
-0.0462619774
-0.0390228592
-0.0426425263
0.00441099843
0.0261281431
-0.0227352157
-0.0154963117
0.0858498588
-0.0426425263
-0.0299743358
0.0152695701
0.00984028447
-0.00644757366
0.000791546132
-0.0136864781
-0.0227352157
-0.0118768597
-0.06074
-0.0571203344
0.000791330414
0.006220832
-0.0372132398
-0.0173061453
0.0261281431
-0.0335937887
0.00803066604
0.0134599525
-0.0625496209
0.0315572135
-0.0100668101
-0.00644757366
-0.0480718128
-0.00101828773


In [12]:
Generator = keras.models.load_model("model/c=10epoch=30/generator.h5")
Discriminator = keras.models.load_model("model/c=10epoch=30/discriminator.h5")
for i in range(50):
    test_data = gen_g_data(10, n_sample)
    gen_data = gen_n_dim_gaussian(10, 1, n_sample)
    input_data = Generator(gen_data)
    sphPoints = inv_sphere_proj(input_data, 9, n_sample, math.sqrt(10))
    fake_test_data = sphPoints + gen_n_dim_gaussian(10, 11, n_sample)
    tf.print((d_metric(test_data, fake_test_data) - 0.989016473) / 0.989016473)

-0.0232720021
-0.0242830347
-0.0333830491
-0.0252941884
-0.030349711
-0.0232720021
-0.0192276333
-0.0374274179
-0.0323720165
-0.0414719097
-0.0323718973
-0.030349711
-0.0323718973
-0.0343940817
-0.0263053402
-0.0414719097
-0.0343940817
-0.0141719887
-0.0343940817
-0.0364163853
-0.0252941884
-0.0343940817
-0.0151831415
-0.0232720021
-0.0323718973
-0.0212498177
-0.0293386783
-0.0263053402
-0.0222609695
-0.030349711
-0.0313608646
-0.0273163728
-0.0232720021
-0.0323720165
-0.0273163728
-0.0182164796
-0.0374274179
-0.0273163728
-0.030349711
-0.0273163728
-0.0222609695
-0.0273163728
-0.0283275247
-0.020238664
-0.0182164796
-0.0263053402
-0.0343940817
-0.0242830347
-0.0445051268
-0.0242830347


In [20]:
import numpy as np
a  = np.random.random((10,5))

In [22]:
a[:,:2]

array([[0.7341332 , 0.63037366],
       [0.71381344, 0.58253878],
       [0.33585669, 0.40557331],
       [0.51888003, 0.49725872],
       [0.75951094, 0.0756263 ],
       [0.84728424, 0.89179385],
       [0.82929944, 0.43424306],
       [0.81637646, 0.43209302],
       [0.39651135, 0.93528132],
       [0.57390997, 0.64628727]])

In [10]:
a * a

array([0.83274888, 0.9939846 , 0.31596909, 0.05447689, 0.0813149 ,
       0.67936586, 0.09360066, 0.73566398, 0.13293953, 0.75883761])

In [12]:
(a*a).sum()

4.678902011095536

In [25]:
a

array([[0.7341332 , 0.63037366, 0.75456397, 0.93032878, 0.26875958],
       [0.71381344, 0.58253878, 0.31487357, 0.34280106, 0.69598909],
       [0.33585669, 0.40557331, 0.35797095, 0.29917566, 0.1613315 ],
       [0.51888003, 0.49725872, 0.1762209 , 0.1434058 , 0.60804367],
       [0.75951094, 0.0756263 , 0.38864956, 0.57877381, 0.25279431],
       [0.84728424, 0.89179385, 0.78106918, 0.22529227, 0.90539604],
       [0.82929944, 0.43424306, 0.48173197, 0.45862155, 0.38851112],
       [0.81637646, 0.43209302, 0.69814316, 0.94512075, 0.66282303],
       [0.39651135, 0.93528132, 0.418282  , 0.31038226, 0.84723648],
       [0.57390997, 0.64628727, 0.1495433 , 0.92642984, 0.91364709]])

In [6]:
import numpy as np

In [7]:
tmp = np.random.random((5, 5))
tmp = np.dot(tmp, tmp.transpose())
sig_G = tmp + tmp.T

In [8]:
sig_G

array([[2.82731935, 1.55988435, 1.45011379, 2.19563161, 2.06688764],
       [1.55988435, 2.16539188, 1.47347009, 2.37951016, 2.3712208 ],
       [1.45011379, 1.47347009, 2.79123566, 1.66814189, 2.40989925],
       [2.19563161, 2.37951016, 1.66814189, 4.20022726, 3.58517219],
       [2.06688764, 2.3712208 , 2.40989925, 3.58517219, 3.56953164]])

In [16]:
sig_G = np.array([[1,0],[0,1]])

In [17]:
from scipy.integrate import dblquad

In [30]:
def gauss(x, sigma):
    return np.exp(-np.dot(np.dot(x, np.linalg.inv(sigma)), x) / 2) / (2 * np.pi) / np.sqrt(np.linalg.det(sigma))

def cal_numerical_TVD(sigma, i, j, c):
    sig = np.array([[sigma[i][i], sigma[i][j]], [sigma[j][i], sigma[j][j]]])
    print(sig)
    val,err = dblquad(lambda y, x: np.abs(gauss(np.array([x,y]), sig)- gauss(np.array([x,y]), c * sig)) / 2, -np.inf, np.inf, -np.inf, np.inf)
    return val

In [25]:
gauss([0,0], sig_G)

0.15915494309189535

In [29]:
dblquad(lambda y, x: np.abs(gauss(np.array([x,y]), sig_G)), -np.inf, np.inf, -np.inf, np.inf)

(1.0000000000000322, 1.3150127836595985e-08)

In [21]:
cal_numerical_TVD(sig_G, 0, 1, 2)

[[1 0]
 [0 1]]


0.49999991116798215

In [22]:
import tensorflow as tf

In [23]:
def g_metric_base(c, n_dim):
    """
    TV distance 解析解

    c:

    n_dim:
        dimension of the data
    """
    r_sq = n_dim * (1 + c) * np.log(1 + c) / c
    return tf.math.igamma(n_dim / 2, r_sq / 2) - tf.math.igamma(n_dim / 2, r_sq / (1 + c) / 2)

In [24]:
g_metric_base(1, 2)

Metal device set to: Apple M1 Max


2022-08-28 23:17:23.786698: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-28 23:17:23.786881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(), dtype=float32, numpy=0.24999994>

In [15]:
import random

In [25]:
random.randint(0, 9)

0

In [28]:
u_G = np.random.random(10)
assert (u_G == 0).sum() != 10
tmp = np.random.random((10, 10))
tmp = np.dot(tmp, tmp.transpose())
sig_G = tmp + tmp.T

In [39]:
test_data = np.random.multivariate_normal(u_G, sig_G, 5)
test_data[:, 1:2], test_data[:, 2:3]

(array([[-2.53525332],
        [-1.49494876],
        [-0.18374018],
        [ 1.16310874],
        [-0.39519577]]),
 array([[-2.93411147],
        [-3.32095802],
        [ 1.95910148],
        [ 1.50253966],
        [ 1.3269711 ]]))

In [40]:
test_data

array([[-1.57885516, -2.53525332, -2.93411147, -1.75207227, -5.79340148,
        -3.46264803, -1.58140696, -3.43348749, -2.59920705, -3.15014966],
       [-3.21835578, -1.49494876, -3.32095802,  0.57149928, -4.14439394,
        -0.38179985, -1.13094273, -2.28764882, -1.06886653,  1.61963383],
       [ 1.66183768, -0.18374018,  1.95910148,  0.50917848,  0.22251862,
         0.36584611, -0.82185373,  1.89565953,  0.19978984,  2.36237118],
       [ 1.36999618,  1.16310874,  1.50253966,  2.04846763,  0.96897398,
         0.54578795,  1.23706717,  1.35277076,  2.41742168,  1.63950555],
       [ 1.12786427, -0.39519577,  1.3269711 ,  1.82747513,  2.75973176,
         0.42076208,  0.19675107,  2.91231783,  2.47538466,  1.32900909]])

In [41]:
test_data = tf.concat([test_data[:, 1:2], test_data[:, 2:3]], 1)

In [42]:
test_data

<tf.Tensor: shape=(5, 2), dtype=float64, numpy=
array([[-2.53525332, -2.93411147],
       [-1.49494876, -3.32095802],
       [-0.18374018,  1.95910148],
       [ 1.16310874,  1.50253966],
       [-0.39519577,  1.3269711 ]])>